# General Stuff

---

In [1]:
import os
import geopandas as gpd
import pandas as pd

import numpy as np
import math
from math import sin, cos, radians, pi

from pyproj import Geod

In [2]:
SCRIPT_PATH = "/Users/rachelralph/Downloads/AI4PublicHealth-main 3"

# Read the road points w/ elevation .shp file via geopandas and store as a pandas dataframe.
ROAD_POINT_WITH_ELEVATION_PATH = os.path.join(SCRIPT_PATH, "Datasets/MZUZU_roads_pointdata_with_elevation.shp")
ROAD_POINT_WITH_ELEVATION_DATA = gpd.read_file(ROAD_POINT_WITH_ELEVATION_PATH)
ROAD_POINT_WITH_ELEVATION_DATA = pd.DataFrame(ROAD_POINT_WITH_ELEVATION_DATA)

BUILDING_FILE_WITH_ELEVATION = os.path.join(SCRIPT_PATH, "Datasets/MZUZU_buildings_with_elevation.shp")
BUILDING_FILE_WITH_ELEVATION = gpd.read_file(BUILDING_FILE_WITH_ELEVATION)
print(BUILDING_FILE_WITH_ELEVATION)
#building_dataframe = pd.DataFrame(BUILDING_FILE_WITH_ELEVATION)
                                                
extra_data = os.path.join(SCRIPT_PATH, "Datasets/MZUZU_roads_lines_CORRECT.shp")
extra_data = gpd.read_file(extra_data)
extra_data = pd.DataFrame(extra_data)
#extra_data['geometry']

           fid       osm_id  name addrfull addrhousen buildingma building  \
0          1.0  464889723.0  None     None       None       None      yes   
1          2.0  464889724.0  None     None       None       None      yes   
2          3.0  464889725.0  None     None       None       None      yes   
3          4.0  464889726.0  None     None       None       None      yes   
4          5.0  464889727.0  None     None       None       None      yes   
...        ...          ...   ...      ...        ...        ...      ...   
42908  42909.0  476236516.0  None     None       None       None      yes   
42909  42910.0  476236518.0  None     None       None       None      yes   
42910  42911.0  476236520.0  None     None       None       None      yes   
42911  42912.0  476236521.0  None     None       None       None      yes   
42912  42913.0  476236523.0  None     None       None       None      yes   

      buildingle addrstreet source office addrcity  longitude  latitude  \


In [3]:
building_dataframe = pd.DataFrame(BUILDING_FILE_WITH_ELEVATION)
for index, row in building_dataframe.iterrows():
    geometry = row

In [4]:
building_dataframe

,fid,osm_id,name,addrfull,addrhousen,buildingma,building,buildingle,addrstreet,source,office,addrcity,longitude,latitude,SAMPLE_1,geometry
0,1.0,464889723.0,None,None,None,None,yes,None,None,None,None,None,34.038,-11.390,1343,POINT (34.03821 -11.38966)
1,2.0,464889724.0,None,None,None,None,yes,None,None,None,None,None,34.037,-11.389,1353,POINT (34.03664 -11.38945)
2,3.0,464889725.0,None,None,None,None,yes,None,None,None,None,None,34.037,-11.390,1353,POINT (34.03673 -11.38952)
3,4.0,464889726.0,None,None,None,None,yes,None,None,None,None,None,34.037,-11.390,1348,POINT (34.03671 -11.38976)
4,5.0,464889727.0,None,None,None,None,yes,None,None,None,None,None,34.037,-11.390,1348,POINT (34.03669 -11.38962)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42908,42909.0,476236516.0,None,None,None,None,yes,None,None,None,None,None,33.977,-11.483,1335,POINT (33.97692 -11.48333)
42909,42910.0,476236518.0,None,None,None,None,yes,None,None,None,None,None,33.976,-11.484,1333,POINT (33.97622 -11.48376)
42910,42911.0,476236520.0,None,None,None,None,yes,None,None,None,None,None,33.976,-11.484,1332,POINT (33.97591 -11.48423)
42911,42912.0,476236521.0,None,None,None,None,yes,None,None,None,None,None,33.976,-11.484,1335,POINT (33.97611 -11.48407)


In [5]:
INCLUDE_MULTILINE = False

def road_line_processing(road_line_df):
    """Clean the .shp file that contains the route data. Create a second pandas data frame to store a processed
        version of the original data from the .shp file. """

    processed_data_line = []


    for rows in range(len(road_line_df.index)):
        coordinates_line = road_line_df.iloc[rows, 11]
        string_type = (type(coordinates_line))

        if INCLUDE_MULTILINE:
            if str(string_type) == "<class 'shapely.geometry.linestring.LineString'>":
                coordinates_line = list(coordinates_line.coords)

                start_longitude_line = coordinates_line[0][0]
                start_latitude_line = coordinates_line[0][1]
                end_longitude_line = coordinates_line[-1][0]
                end_latitude_line = coordinates_line[-1][1]
                
                all_coor = []
                
                for coordinate in coordinate_line:
                        all_coor.append(coordinate) 

                processed_data_line.append(
                    (start_longitude_line, start_latitude_line, end_longitude_line, end_latitude_line))

            elif str(string_type) != "<class 'shapely.geometry.linestring.MultiLineString'>":
                for item in coordinates_line:
                    coordinates_line = item

                    coordinates_line = list(coordinates_line.coords)

                    start_longitude_line = coordinates_line[0][0]
                    start_latitude_line = coordinates_line[0][1]
                    end_longitude_line = coordinates_line[-1][0]
                    end_latitude_line = coordinates_line[-1][1]
                    
                    all_coor = []
                    
                    for coordinate in coordinate_line:
                        all_coor.append(coordinate)

                    processed_data_line.append(
                        (start_longitude_line, start_latitude_line, end_longitude_line, end_latitude_line, all_coor))

            else:
                print("There is a unique string type that is neither LineString or MultiString:")
                print("    ", string_type)

        else:
            if str(string_type) == "<class 'shapely.geometry.linestring.LineString'>":
                coordinates_line = list(coordinates_line.coords)

                start_longitude_line = round(coordinates_line[0][0], 6)
                start_latitude_line = round(coordinates_line[0][1], 6)
                end_longitude_line = round(coordinates_line[-1][0], 6)
                end_latitude_line = round(coordinates_line[-1][1], 6)
                
                all_coor = []
                
                for coordinate in coordinates_line:
                        all_coor.append(coordinate)

                processed_data_line.append(
                    (start_longitude_line, start_latitude_line, end_longitude_line, end_latitude_line, all_coor))

            else:
                continue

    processed_data_line = pd.DataFrame(processed_data_line)
    processed_data_line = processed_data_line.rename(
        columns={0: "Start Longitude", 1: "Start Latitude", 2: "End Longitude", 3: "End Latitude", 4:"Coordinates List"})

    return processed_data_line

road_line_nodes = road_line_processing(extra_data)
road_line_nodes

,Start Longitude,Start Latitude,End Longitude,End Latitude,Coordinates List
0,33.985349,-11.390521,33.984215,-11.369368,"[(33.9853493, -11.3905214), (33.9853064, -11.3..."
1,33.957398,-11.397216,34.048045,-11.394335,"[(33.95739758409233, -11.3972159503597), (33.9..."
2,33.960214,-11.390768,33.962719,-11.399591,"[(33.96021380982741, -11.390768323977507), (33..."
3,33.989335,-11.392850,33.992700,-11.409748,"[(33.9893353, -11.3928496), (33.989345, -11.39..."
4,34.046345,-11.387836,34.044863,-11.388096,"[(34.04634539527059, -11.387836474070907), (34..."
...,...,...,...,...,...
5458,33.970628,-11.449207,33.970089,-11.448832,"[(33.9706278, -11.4492074), (33.9705393, -11.4..."
5459,33.969799,-11.450311,33.969593,-11.450297,"[(33.9697988, -11.4503106), (33.9695931, -11.4..."
5460,33.964466,-11.436791,33.963564,-11.436120,"[(33.9644656, -11.4367907), (33.9644047, -11.4..."
5461,33.967368,-11.445715,33.964825,-11.444966,"[(33.9673681, -11.4457151), (33.9671929, -11.4..."


In [6]:
first_run = True

print(road_line_nodes.iat[0, 4])

node_data = []
for index, row in road_line_nodes.iterrows():
    for i in range(len(row["Coordinates List"])):
        connections = []
        coordinate_pair = row["Coordinates List"][i]
        if i != 0: 
            connections.append(row["Coordinates List"][i - 1])
        if i != len(row["Coordinates List"]) - 1:
            connections.append(row["Coordinates List"][i + 1])
        node_data.append([coordinate_pair, connections])
    
    
node_data = pd.DataFrame(node_data)         
node_data

[(33.9853493, -11.3905214), (33.9853064, -11.3898512), (33.9852493, -11.3889611), (33.985201, -11.388167), (33.9851635, -11.3876542), (33.9851416, -11.3873689), (33.9850074, -11.3858701), (33.9849922, -11.3856728), (33.984924, -11.3845922), (33.9848334, -11.3835482), (33.9847914, -11.3830646), (33.9847394, -11.3824659), (33.9846665, -11.3815285), (33.9846399, -11.3809951), (33.9846211, -11.3802556), (33.9846279, -11.3800459), (33.9846433, -11.3797074), (33.9847506, -11.3794077), (33.9848867, -11.3791335), (33.9850564, -11.3788239), (33.9851911, -11.3786017), (33.9852354, -11.3785287), (33.9858791, -11.3773704), (33.9860622, -11.3770128), (33.9860762, -11.3769908), (33.986246, -11.3767223), (33.9866456, -11.3760517), (33.9873811, -11.3748211), (33.9874798, -11.3746785), (33.9875616, -11.3744438), (33.9876065, -11.3741953), (33.9875528, -11.3735642), (33.9873965, -11.3733322), (33.9873125, -11.3732074), (33.9872518, -11.3731172), (33.987111, -11.3729082), (33.9866302, -11.3722862), (33.9

,0,1
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512)]"
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9852493, -11.3..."
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38..."
3,"(33.985201, -11.388167)","[(33.9852493, -11.3889611), (33.9851635, -11.3..."
4,"(33.9851635, -11.3876542)","[(33.985201, -11.388167), (33.9851416, -11.387..."
...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]"
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4..."
66013,"(33.9691152, -11.450674)","[(33.9689286, -11.4507671), (33.9694754, -11.4..."
66014,"(33.9694754, -11.4504237)","[(33.9691152, -11.450674), (33.9695931, -11.45..."


In [7]:
lat_long_pairs = {}

multiple_index_pairs = []

for index, row in node_data.iterrows():
    lat_long_coor = row[0]
    if lat_long_coor in lat_long_pairs.keys():
        lat_long_pairs[lat_long_coor].append(index)
        if lat_long_coor not in multiple_index_pairs:
            multiple_index_pairs.append(lat_long_coor)
    else: 
        lat_long_pairs[lat_long_coor] = [index]

print(len(lat_long_pairs.keys()))

node_data

57880


,0,1
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512)]"
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9852493, -11.3..."
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38..."
3,"(33.985201, -11.388167)","[(33.9852493, -11.3889611), (33.9851635, -11.3..."
4,"(33.9851635, -11.3876542)","[(33.985201, -11.388167), (33.9851416, -11.387..."
...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]"
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4..."
66013,"(33.9691152, -11.450674)","[(33.9689286, -11.4507671), (33.9694754, -11.4..."
66014,"(33.9694754, -11.4504237)","[(33.9691152, -11.450674), (33.9695931, -11.45..."


In [8]:
node_data[2] = True
node_data

,0,1,2
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512)]",True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9852493, -11.3...",True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",True
3,"(33.985201, -11.388167)","[(33.9852493, -11.3889611), (33.9851635, -11.3...",True
4,"(33.9851635, -11.3876542)","[(33.985201, -11.388167), (33.9851416, -11.387...",True
...,...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]",True
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",True
66013,"(33.9691152, -11.450674)","[(33.9689286, -11.4507671), (33.9694754, -11.4...",True
66014,"(33.9694754, -11.4504237)","[(33.9691152, -11.450674), (33.9695931, -11.45...",True


In [9]:
min_length = math.inf
for pair in multiple_index_pairs:
    if pair == (33.9861071, -11.3907319):
        print("Processing 106..")
    index_list = lat_long_pairs[pair]
    if len(index_list) < min_length: 
        min_length = len(index_list)
    first_index = index_list[0]
    for second_index in index_list[1:]:
        all_connections = list(set(node_data.at[first_index, 1] + node_data.at[second_index, 1]))
        node_data.at[first_index, 1] = all_connections
        node_data.at[second_index, 2] = False
        lat_long_pairs[pair] = [first_index]
    if pair == (33.9861071, -11.3907319):
        print(all_connections)
print(min_length)
print(lat_long_pairs[(33.9861071, -11.3907319)])
node_data

Processing 106..
[(33.9857075, -11.3906313), (33.9860044, -11.3908575), (33.9866929, -11.3908778)]
2
[105]


,0,1,2
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512), (33.9857075, -11.3...",True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9852493, -11.3...",True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",True
3,"(33.985201, -11.388167)","[(33.9852493, -11.3889611), (33.9851635, -11.3...",True
4,"(33.9851635, -11.3876542)","[(33.985201, -11.388167), (33.9851416, -11.387...",True
...,...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]",False
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",True
66013,"(33.9691152, -11.450674)","[(33.9689286, -11.4507671), (33.9694754, -11.4...",True
66014,"(33.9694754, -11.4504237)","[(33.9691152, -11.450674), (33.9695931, -11.45...",True


In [10]:
node_data[3] = False
node_data
print(node_data.at[106, 1])

[(33.9861071, -11.3907319), (33.9868811, -11.390917), (33.9866218, -11.3912735)]


In [11]:
for index, row in node_data.iterrows():
    if len(row[1]) != 2:
        node_data.iat[index, 3] = True
node_data

,0,1,2,3
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512), (33.9857075, -11.3...",True,True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9852493, -11.3...",True,True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",True,False
3,"(33.985201, -11.388167)","[(33.9852493, -11.3889611), (33.9851635, -11.3...",True,False
4,"(33.9851635, -11.3876542)","[(33.985201, -11.388167), (33.9851416, -11.387...",True,True
...,...,...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]",False,True
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",True,False
66013,"(33.9691152, -11.450674)","[(33.9689286, -11.4507671), (33.9694754, -11.4...",True,False
66014,"(33.9694754, -11.4504237)","[(33.9691152, -11.450674), (33.9695931, -11.45...",True,False


In [12]:

print(node_data.at[8386, 1])

[(33.9885659, -11.4309689), (33.9887352, -11.4310535)]


In [13]:
import copy 
node_data_copy = node_data
import sys
for index, row in node_data_copy.iterrows():
    if row[2]:
        if row[3] == True: 
            continue
        else:
            if(len(node_data.at[index, 1]) >= 2):
                connection1 = node_data_copy.at[index, 1][0]
                connection2 = node_data_copy.at[index, 1][1]
                index1 = lat_long_pairs[connection1][0]
                index2 = lat_long_pairs[connection2][0]
                node_data_copy.at[index1, 1].remove(node_data_copy.at[index, 0])
                node_data_copy.at[index2, 1].remove(node_data_copy.at[index, 0])
                node_data_copy.at[index1, 1] = list(set(node_data_copy.at[index1, 1] + [node_data_copy.at[index2, 0]]))
                node_data_copy.at[index2, 1] = list(set([node_data_copy.at[index1, 0]] + node_data_copy.at[index2, 1]))

node_data_copy.reset_index().drop("index", axis = 1)



,0,1,2,3
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512), (33.9861071, -11.3...",True,True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9851635, -11.3...",True,True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",True,False
3,"(33.985201, -11.388167)","[(33.9853064, -11.3898512), (33.9851635, -11.3...",True,False
4,"(33.9851635, -11.3876542)","[(33.9853064, -11.3898512), (33.9849922, -11.3...",True,True
...,...,...,...,...
66011,"(33.9687616, -11.4507767)","[(33.9689286, -11.4507671)]",False,True
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",True,False
66013,"(33.9691152, -11.450674)","[(33.9694754, -11.4504237), (33.9687616, -11.4...",True,False
66014,"(33.9694754, -11.4504237)","[(33.9695931, -11.4502973), (33.9687616, -11.4...",True,False


In [14]:
index_list = []
for index, row in node_data_copy.iterrows():
    if row[2] == False:
        index_list.append(index)

for index in index_list: 
    node_data_copy = node_data_copy.drop(index, axis = 0)
node_data_copy

,0,1,2,3
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512), (33.9861071, -11.3...",True,True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9851635, -11.3...",True,True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",True,False
3,"(33.985201, -11.388167)","[(33.9853064, -11.3898512), (33.9851635, -11.3...",True,False
4,"(33.9851635, -11.3876542)","[(33.9853064, -11.3898512), (33.9849922, -11.3...",True,True
...,...,...,...,...
66008,"(33.9652771, -11.4451223)","[(33.9650089, -11.4449751), (33.9673681, -11.4...",True,False
66009,"(33.9650089, -11.4449751)","[(33.9648248, -11.4449655), (33.9673681, -11.4...",True,False
66012,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",True,False
66013,"(33.9691152, -11.450674)","[(33.9694754, -11.4504237), (33.9687616, -11.4...",True,False


In [15]:
node_data_copy = node_data_copy.reset_index()
node_data_copy = node_data_copy.drop("index", axis = 1)
node_data_copy = node_data_copy.drop(2, axis = 1)

In [16]:
node_data_copy = node_data_copy.rename(
        columns={0: "Long/Lat Coordinates", 1: "Connections", 3: "Is Intersection", })

node_data_copy

,Long/Lat Coordinates,Connections,Is Intersection
0,"(33.9853493, -11.3905214)","[(33.9853064, -11.3898512), (33.9861071, -11.3...",True
1,"(33.9853064, -11.3898512)","[(33.9853493, -11.3905214), (33.9851635, -11.3...",True
2,"(33.9852493, -11.3889611)","[(33.9853064, -11.3898512), (33.985201, -11.38...",False
3,"(33.985201, -11.388167)","[(33.9853064, -11.3898512), (33.9851635, -11.3...",False
4,"(33.9851635, -11.3876542)","[(33.9853064, -11.3898512), (33.9849922, -11.3...",True
...,...,...,...
57875,"(33.9652771, -11.4451223)","[(33.9650089, -11.4449751), (33.9673681, -11.4...",False
57876,"(33.9650089, -11.4449751)","[(33.9648248, -11.4449655), (33.9673681, -11.4...",False
57877,"(33.9689286, -11.4507671)","[(33.9687616, -11.4507767), (33.9691152, -11.4...",False
57878,"(33.9691152, -11.450674)","[(33.9694754, -11.4504237), (33.9687616, -11.4...",False


In [17]:
# node_data_copy.to_csv("node_data.csv", encoding = "utf-8", index = False)

In [18]:
class Node():
    
    def __init__(self, road_condition, elevation, long_lat, connections):
        self.road_condition = road_condition
        self.elevation = elevation
        self.connections = connections
        self.long_lat = long_lat
        self.f = None
        self.g = None
        self.h = None
        self.id = 10001
        
    def convert_connections_for_graph(self, connections): 
        self.connections = connections

In [19]:
node_dictionary = {}
for index, row in node_data_copy.iterrows():
    if row["Is Intersection"]:
        row_node = Node(None, None, row["Long/Lat Coordinates"], row["Connections"])
        node_dictionary[row["Long/Lat Coordinates"]] = row_node

print(node_dictionary.keys())
    

dict_keys([(33.9853493, -11.3905214), (33.9853064, -11.3898512), (33.9851635, -11.3876542), (33.9849922, -11.3856728), (33.984924, -11.3845922), (33.9848334, -11.3835482), (33.9847394, -11.3824659), (33.9846665, -11.3815285), (33.9846211, -11.3802556), (33.9846433, -11.3797074), (33.9848867, -11.3791335), (33.9851911, -11.3786017), (33.9860762, -11.3769908), (33.9866456, -11.3760517), (33.9874798, -11.3746785), (33.9873965, -11.3733322), (33.9873125, -11.3732074), (33.9872518, -11.3731172), (33.9853963, -11.3708006), (33.98421488961864, -11.369367531520561), (33.95739758409233, -11.3972159503597), (33.9614824, -11.3944941), (33.9622054, -11.3940925), (33.9655164, -11.3924521), (33.9693732, -11.3922026), (33.9715848, -11.3917973), (33.972729, -11.3919268), (33.9753498, -11.3925172), (33.9759196, -11.3926953), (33.9769758, -11.3926839), (33.9774153, -11.3925905), (33.9792891, -11.3914398), (33.9799044, -11.3908885), (33.9806119, -11.3904287), (33.9813736, -11.3900286), (33.9825525, -11.3

In [20]:
def heuristic(long1, lat1, long2, lat2):
    return math.sqrt((lat2 - lat1)**2 + (long2 - long1)**2)

In [21]:
from operator import attrgetter
def node_to_node_search(start_node, goal):
    open_list = [start_node]
    closed_list = []
    final_route = []

    start_node.f = 0
    start_node.g = 0
    tree_dict = {start_node: 0}

    while len(open_list) != 0:
        current_node = open_list[0]

        # Set the new node to be the previous sub_node with the lowest f(x).
        if current_node != start_node:
            current_node = min(open_list, key=attrgetter('f'))

        #
        level = tree_dict[current_node]
        open_list.remove(current_node)

        if level == len(final_route):
            final_route.append(current_node)
        else:
            final_route[level] = current_node

        # Add the current node to the checked list
        closed_list.append(current_node)

        # Check if the goal node has been reached
        if current_node == goal:
            return current_node.f

        # For the current_node, go through all of its connections, and determine their f(x) as a sum of their h(x)
        # and g(x).
        for sub_node in current_node.connections:
            long_lat = sub_node
            if sub_node in node_dictionary.keys():
                sub_node = node_dictionary[sub_node]
            else: 
                continue
            if sub_node in closed_list:
                continue
            found = False
            for key, value in tree_dict.items():
                if key == sub_node:
                    found = True
            if not found:
                tree_dict[sub_node] = level + 1

            sub_node.g = heuristic(current_node.long_lat[0], current_node.long_lat[1], sub_node.long_lat[0], sub_node.long_lat[1]) + current_node.g
            sub_node.h = heuristic(sub_node.long_lat[0], sub_node.long_lat[1], goal.long_lat[0], goal.long_lat[1]) * 4146.282847732093
            sub_node.f = sub_node.g + sub_node.h

            open_list.append(sub_node)

        


In [22]:
import random
house_ids = []

random.seed(42)
for i in range(10):
    house_ids.append(random.randint(0, len(building_dataframe)))

nearest_intersections = []

for house_id in house_ids: 
    print(house_id)
    coordinates = list(building_dataframe.loc[house_id, "geometry"].coords)
    long = coordinates[0][0] # Contains max 7 decimal points
    lat = coordinates[0][1] # Contains max 7 decimal points
    print(long, lat)
    elevation = building_dataframe.at[house_id, "SAMPLE_1"] 
    near_intersection = None
    near_intersection_dist = math.inf
    for index, row in node_data_copy.iterrows():
        if row["Is Intersection"]:
            node_long = row["Long/Lat Coordinates"][0]
            node_lat = row["Long/Lat Coordinates"][1]
            if near_intersection_dist > heuristic(lat, long, node_lat, node_long):
                near_intersection_dist = heuristic(lat, long, node_lat, node_long)
                near_intersection = node_dictionary[row["Long/Lat Coordinates"]]
    nearest_intersections.append(near_intersection)
        

41905
33.97678378356073 -11.445550750515714
7296
34.02786745 -11.43902875
1639
33.99958780243911 -11.425001379825916
18024
34.00415347448529 -11.441856163968605
16049
34.01186065 -11.45268045
14628
34.0109073 -11.4546102
9144
34.0257843 -11.435348800000002
6717
34.01536025000001 -11.435784
35741
34.020120405630344 -11.446162479034573
5697
34.02420459590668 -11.435904271125944


In [23]:
nearest_intersections

In [24]:
for node in nearest_intersections: 
    print(node.long_lat)

(33.97714, -11.4458369)
(34.0277206, -11.4396859)
(33.9998205, -11.4249803)
(34.0035481, -11.4417975)
(34.0115509, -11.4520812)
(34.0109894, -11.4546968)
(34.0252915, -11.4352003)
(34.0152356, -11.4358103)
(34.0203304, -11.4463785)
(34.0241398, -11.4357616)


In [25]:
graph = {}
for intersection in nearest_intersections:
    houses_completed = []
    houses_visited = []
    for i in range(3):
        smallest_dist = math.inf
        closest_house = None
        for house_to_go in nearest_intersections:
            if house_to_go not in houses_completed and house_to_go != intersection:
                dist = heuristic(house_to_go.long_lat[0], house_to_go.long_lat[1], intersection.long_lat[0], intersection.long_lat[1])
                if  dist < smallest_dist:
                    smallest_dist = dist
                    closest_house = house_to_go
        cost = node_to_node_search(closest_house, intersection)
        houses_completed.append(closest_house)
        houses_visited.append((closest_house, cost))
        graph[intersection] = houses_visited

            
                
for key in graph.keys():
    print(key.long_lat, graph[key][0][0].long_lat, graph[key][1][0].long_lat, graph[key][2][0].long_lat)
    print(key.long_lat, graph[key][0][1], graph[key][1][1], graph[key][2][1])
        

(33.97714, -11.4458369) (34.0035481, -11.4417975) (33.9998205, -11.4249803) (34.0115509, -11.4520812)
(33.97714, -11.4458369) 0.03501622802094387 0.04081484395970795 0.051469053386925465
(34.0277206, -11.4396859) (34.0252915, -11.4352003) (34.0241398, -11.4357616) (34.0203304, -11.4463785)
(34.0277206, -11.4396859) 0.007942932816399791 0.007734848265428463 0.013480875105215341
(33.9998205, -11.4249803) (34.0035481, -11.4417975) (34.0152356, -11.4358103) (34.0241398, -11.4357616)
(33.9998205, -11.4249803) 0.025920108761399985 0.024420617628263086 0.033678242760335673
(34.0035481, -11.4417975) (34.0115509, -11.4520812) (34.0152356, -11.4358103) (34.0109894, -11.4546968)
(34.0035481, -11.4417975) 0.015467868102380036 0.03521346080790205 0.021663055067509727
(34.0115509, -11.4520812) (34.0109894, -11.4546968) (34.0203304, -11.4463785) (34.0035481, -11.4417975)
(34.0115509, -11.4520812) 0.003713961936154437 0.017756915662427597 0.014278698808105684
(34.0109894, -11.4546968) (34.0115509, -11

In [26]:
nodes_list = graph.keys()

nodes_for_graph = []
id_num = 0

for node in nodes_list: 
    new_node = Node(None, None, node.long_lat, None)
    new_node.id = id_num
    connections = {}
    for house_cost in graph[node]:
        #print(house_cost[0], house_cost[1])
        connections[house_cost[0]] = house_cost[1]
        #print(connections[house_cost[0]])
    new_node.convert_connections_for_graph(connections)
    nodes_for_graph.append(new_node)
    id_num += 1
print(nodes_for_graph[0].connections)

for key in nodes_for_graph[0].connections.keys():
    print(key.long_lat)

{<__main__.Node object at 0x7fc136ae57f0>: 0.03501622802094387, <__main__.Node object at 0x7fc13678e250>: 0.04081484395970795, <__main__.Node object at 0x7fc136aeae50>: 0.051469053386925465}
(34.0035481, -11.4417975)
(33.9998205, -11.4249803)
(34.0115509, -11.4520812)


In [27]:
class Graph():
    
    def __init__(self, nodes_for_graph):
        self.nodes = nodes_for_graph
    
    def convert_to_matrix(self):
        matrix = np.zeros((len(self.nodes), len(self.nodes)))
        matrix = np.where(matrix == 0, math.inf, matrix)
        for node in self.nodes:
            for connections in node.connections:
                matrix[node.id][connections.id] = node.connections[connections]
        return matrix
   
    def size(self):
        return len(self.nodes)


In [28]:
class PriorityQueue:
    def __init__(self):
        self.queue = {}

    def isEmpty(self):
        return len(self.queue) == 0

    def push_with_matrix(self, node, priority, matrix):
        self.queue[node] = [priority, matrix]

    def push_wo_matrix(self, node, priority):
        self.queue[node] = priority

    def inQueue(self, node):
        for node in self.queue:
            if node.id == node.id:
                return True
        return False

    def pop_with_matrix(self):
        node = min(self.queue, key=attrgetter('cost'))
        cost = self.queue[node][0]
        matrix = self.queue[node][1]

        del self.queue[node]
        return node, cost, matrix

    def peek(self):
        node = min(self.queue, key=attrgetter('cost'))
        cost = self.queue[node]
        return node, cost

    def pop_wo_matrix(self):
        node = min(self.queue, key=attrgetter('cost'))
        cost = self.queue[node]
        del self.queue[node]
        return node, cost

In [29]:
class SolidStateNode:

    def __init__(self, parent_node, node_id, node, level, reduced_matrix, cost):
        self.node = node
        self.node_id = node_id
        self.reduced_matrix = reduced_matrix
        self.parent_node = parent_node
        self.level = level
        self.cost = cost
        self.children = []
        if parent_node is not None:
            parent_node.add_children(self)
            self.closedList = parent_node.closedList + [parent_node.node]

        else:
            self.closedList = []

    def add_children(self, child):
        self.children.append(child)


In [30]:
class SolidStateTree:

    def __init__(self, root):
        self.root = root


In [31]:
def calculate_cost(matrix):
    """Calculates the reduction cost of the matrix. Returns the adjacency matrix (reduced matrix) and the associated
    cost. """
    cost = 0

    def row_reduction():
        """Reduce the rows. Each row is reduced based on the lowest number in itself. Returns the row reduced matrix."""

        # Get minimum values of all the rows. Note that this includes the zeros in the rows.
        min_values = np.min(matrix, axis=1)

        # TODO: There has to be a more efficient way of doing this than going through two for loops.
        # Iterate through all of the rows and columns and subtract the row-respective min_value from all of the
        # numbers in the respective row.
        for row in range(len(matrix)):
            for column in range(len(matrix[0])):
                if min_values[row] != math.inf and matrix[row][column] != math.inf:
                    matrix[row][column] -= min_values[row]

        return matrix, min_values

    def col_reduction():
        """Reduce the columns. Each column is reduced based on the lowest number in itself. Returns the column
        reduced matrix. """

        # Get minimum values of all the columns. Note that this includes the zeros in the columns.
        min_values = np.min(matrix, axis=0)

        # TODO: There has to be a more efficient way of doing this than going through two for loops.
        # Iterate through all of the columns and rows and subtract the column-respective min_value from all of the
        # numbers in the respective column.
        for column in range(len(matrix[0])):
            for row in range(len(matrix)):
                if min_values[row] != math.inf and matrix[row][column] != math.inf:
                    matrix[row][column] -= min_values[column]

        return matrix, min_values

    matrix, min_row = row_reduction()
    matrix, min_col = col_reduction()

    # Iterate over the min_row and min_column lists and add each value within them to the cost variable.
    for index in range(len(min_row)):
        if min_row[index] != math.inf:
            cost += min_row[index]
        if min_col[index] != math.inf:
            cost += min_col[index]

    return cost, matrix

In [32]:
def explore_edge(start_node, node_from, node_to, matrix):
    """Sets the node_from row, node_to column, and the point (node_to, node_from) to math.inf. Returns the resulting
    matrix. """

    # Set rows to math.inf.
    matrix[node_from, :] = math.inf

    # Set columns to math.inf.
    matrix[:, node_to] = math.inf

    # Set (j,i) to math.inf.
    matrix[node_to, start_node] = math.inf

    return matrix

In [33]:
def search_solid_state_tree(root):
    if root.parent_node is not None:
        print("Parent: ", root.parent_node.node_id, "ID: ", root.node_id, "House: ", root.node.id)
    else:
        print("ID: ", root.node_id, "House: ", root.node.id)

    for child in root.children:
        search_solid_state_tree(child)


In [34]:
def space_state_algorithm(start_node, original_matrix, graph):
    original_matrix_copy = copy.deepcopy(original_matrix)

    # Reduce the original matrix.
    initial_cost, cost_matrix = calculate_cost(original_matrix_copy)

    priority_queue = PriorityQueue()
    parent_node = graph.nodes[start_node]
    closed_list = []
    visual_list = []
    final_list = []

    root = SolidStateNode(None, 0, parent_node, 0, cost_matrix, initial_cost)
    SolidStateTree(root)

    priority_queue.push_wo_matrix(root, initial_cost)

    i = 1
    num_of_backtracks = 0

    lowest_branch_cost = math.inf
    lowest_branch = []
    first_branch = []
    first_branch_cost = math.inf
    return_val = []

    while not priority_queue.isEmpty():
        search_solid_state_tree(root)
        print("Hey! Checking priority queue...")

        parent_state, cost = priority_queue.pop_wo_matrix()
        # print(parent_state)

        parent = parent_state.node
        parent_matrix = parent_state.reduced_matrix

        # print("PARENT LEVEL: " , parent_state.level)
        if parent_state.level >= graph.size() - 1 + num_of_backtracks:
            if len(first_branch) == 0:
                first_branch_cost = parent_state.cost
                next_state = parent_state
                while next_state != None:
                    first_branch.append(next_state.node.id)
                    next_state = next_state.parent_node
                first_branch.reverse()

            root_connection = False
            for connection in parent.connections:
                if connection.id == root.node.id:
                    root_connection = True

            if not root_connection:
                continue


            parent_matrix_copy = copy.deepcopy(parent_matrix)

            parent_copy_explored = explore_edge(parent.id, parent.id, root.node.id, parent_matrix_copy)
            cost_for_step, parent_copy_reduced = calculate_cost(parent_copy_explored)

            branch_cost = cost_for_step
            branch_cost += cost
            branch_cost += cost_matrix[parent.id, root.node.id]

            if not priority_queue.isEmpty():
                next_node, next_node_cost = priority_queue.peek()



                if branch_cost <= next_node_cost:
                    if branch_cost < first_branch_cost*2:
                        while parent_state is not None:
                            return_val.append(parent_state.node.id)
                            parent_state = parent_state.parent_node

                        return_val.reverse()
                        return_val.append(0)
                        break

                    else:
                        return_val = first_branch
                        break

            
            if priority_queue.isEmpty():
                if first_branch_cost*2 < lowest_branch_cost:
                    print("Returning first branch...")
                    return_val = first_branch
                else:
                    return_val = lowest_branch

            if lowest_branch_cost > branch_cost:
                lowest_branch_cost = branch_cost
                lowest_branch = []
                lowest_state_node = parent_state
                lowest_node = parent
                while lowest_state_node is not None:
                    lowest_branch.append(lowest_state_node.node.id)
                    lowest_state_node = lowest_state_node.parent_node

                lowest_branch.reverse()
                lowest_branch.append(0)

            print("Going to the top of the loop...")

            continue

        if cost > lowest_branch_cost:
            return_val = lowest_branch
            print("Found lowest cost")
            break
        print("Going on down...")

        if len(parent.connections) == 1:
            print("Backtracking...")
            num_of_backtracks = 1

        for sub_node in parent.connections:
            if sub_node not in parent_state.closedList or len(parent.connections) == 1:
                print(sub_node.id)
                # Set initial cost to that of the parent cost.
                total_cost = cost
                parent_matrix_copy = copy.deepcopy(parent_matrix)

                # Add the cost of the edge to the total_cost. The edge cost must be retrieved from the starting node
                # reduced cost matrix (cost_matrix).
                edge_cost = cost_matrix[parent.id, sub_node.id]
                total_cost += edge_cost

                # Add the cost of the lower bound starting at the sub_node. This means that we need to add the
                # math.inf values, then perform a row and column reduction, and add the resulting cost to our total
                # cost. NOTE: This must be done on the current parent reduced matrix.
                parent_matrix_copy_explored = explore_edge(start_node, parent.id, sub_node.id, parent_matrix_copy)
                cost_for_step, parent_matrix_copy_explored_reduced = calculate_cost(parent_matrix_copy_explored)
                total_cost += cost_for_step

                sub_node.cost = total_cost

                state_node = SolidStateNode(parent_state, i, sub_node, parent_state.level + 1,
                                            parent_matrix_copy_explored, total_cost)
                i += 1

                priority_queue.push_wo_matrix(state_node, cost)

        closed_list.append(parent)

    if len(return_val) > 0:
        parent_index = 0

        for item in return_val:
            final_list.append(item)

            if len(final_list) >= 2:
                visual_list.append(
                    [final_list[parent_index], item, original_matrix[final_list[parent_index]][item]])
                parent_index += 1

    else:
        matrix_insert = 0
        for item in closed_list:
            if len(closed_list) - len(final_list) < len(closed_list):
                visual_list[matrix_insert].insert(1, item.id)
                matrix_insert += 1

            final_list.append(item.id)

            if len(closed_list) - len(final_list) == 0:
                visual_list[matrix_insert].insert(1, visual_list[matrix_insert][0])

    return final_list, visual_list


In [35]:
graph = Graph(nodes_for_graph)

for node in graph.nodes:
    print(node.id)
matrix = graph.convert_to_matrix()
final_list, _ = space_state_algorithm(0, matrix, graph)

0
1
2
3
4
5
6
7
8
9


IndexError: index 10001 is out of bounds for axis 0 with size 10

In [ ]:
for index, row in road_line_nodes.iterrows(): 
    if row["Start Latitude"] == -11.464411:
        print(index)
        print(row)

# True values for 37757 <- From point data
# Longitude           33.978497
# Latitude           -11.464378
        
# Predicted values of 37758 from (long, lat, dist, angle) of 37757
# To Longitude        33.978497
# To Latitude        -11.464378
    
# True values for 37758 <- From point data
# Longitude           33.978517
# Latitude           -11.464411

# Create DataFrame

---

In [ ]:
test = 1

radius_earth = 6378.137

def road_elevation_processing(road_elevation_df):
    """Clean the .shp file that contains the route data. Create a second pandas data frame to store a processed
        version of the original data from the .shp file. """

    # Create a secondary pandas data frame that contains the index of nodes, start/end longitude and latitude,
    # elevation, road condition, and road type.
    processed_data = []

    
    for rows in range(len(road_elevation_df.index)):
        
        # Get the coordinates.
        coordinates = list(road_elevation_df.iloc[rows, 22].coords)
        start_latitude = coordinates[0][1] # Contains max 7 decimal points
        start_longitude = coordinates[0][0] # Contains max 7 decimal points

        # Get the elevation, road condition, and road type.
        elevation = road_elevation_df.iloc[rows, 17]
        road_condition = road_elevation_df.iloc[rows, 10]
        road_type = road_elevation_df.iloc[rows, 9]
        
        # Convert distance and angle to connecting node coords. Note the rounding is to 7 decimal points.
        distance = road_elevation_df.iloc[rows, 15] # Contains max 14 decimal points
        angle = road_elevation_df.iloc[rows, 16]# Contains max 14 decimal points
            
        if test == 1:
            radians = math.radians(angle) # Contains max 16 decimal points


            # Convert the start_longitude and latitude to radians.
            from_long = math.radians(start_longitude) # Contains max 16 data points
            from_lat = math.radians(start_latitude)# Contains 17 data points

            # print(len(str(distance)), len(str(angle)), len(str(radians)), len(str(from_long)), len(str(from_lat)))

            # to_lat has 17 decimal points. to_long has 16 decimal points
            to_lat = math.asin(sin(from_lat) * 
                               cos(distance / radius_earth) + 
                               cos(from_lat) * 
                               sin(distance / radius_earth) * 
                               math.cos(radians))
            to_long = from_long + math.atan2(sin(radians) * sin(distance / radius_earth) * cos(from_lat), 
                                             cos(distance / radius_earth) - sin(from_lat) * sin(to_lat))

            to_lat = math.degrees(to_lat) # + 0.0000428
            to_long = math.degrees(to_long) # + (-0.0000254)
    
        else:
            geod = Geod(ellps="WGS84")
            
            to_long, to_lat, _ = geod.fwd(start_longitude, start_latitude, angle, distance, radians = False)
        
        start_longitude = round(start_longitude, 7)
        start_latitude = round(start_latitude, 7)
        to_lat = round((to_lat), 7)
        to_long = round(to_long, 7)

        processed_data.append((start_longitude, start_latitude, elevation, distance, angle, to_long, to_lat, road_condition, road_type, [], []))

    processed_data = pd.DataFrame(processed_data)

    processed_data = processed_data.rename(
        columns={0: "Longitude", 1: "Latitude", 2: "Elevation", 3: "Distance", 4: "Angle", 5: "To Longitude", 6: "To Latitude", 7: "Road Condition", 8: "Road Type", 9: "Connection(s)"})

    return processed_data


road_elevation_nodes = road_elevation_processing(ROAD_POINT_WITH_ELEVATION_DATA)

road_elevation_nodes

# Check that the math is correct

---

In [ ]:
# Test = 1, dp = 7
lat1 = road_elevation_nodes.loc[37758, "Latitude"]
long1 = road_elevation_nodes.loc[37758, "Longitude"]
lat2 = road_elevation_nodes.loc[37757, "To Latitude"]
long2 = road_elevation_nodes.loc[37757, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

In [ ]:
# Test = 2, dp = 7, 37757-37758
lat1 = road_elevation_nodes.loc[37758, "Latitude"]
long1 = road_elevation_nodes.loc[37758, "Longitude"]
lat2 = road_elevation_nodes.loc[37757, "To Latitude"]
long2 = road_elevation_nodes.loc[37757, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

In [ ]:
# Test = 1, dp = 7, 51658-51659
lat1 = road_elevation_nodes.loc[51659, "Latitude"]
long1 = road_elevation_nodes.loc[51659, "Longitude"]
lat2 = road_elevation_nodes.loc[51658, "To Latitude"]
long2 = road_elevation_nodes.loc[51658, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

In [ ]:
# Test = 2, dp = 7, 51658-51659
lat1 = road_elevation_nodes.loc[51659, "Latitude"]
long1 = road_elevation_nodes.loc[51659, "Longitude"]
lat2 = road_elevation_nodes.loc[51658, "To Latitude"]
long2 = road_elevation_nodes.loc[51658, "To Longitude"]

print(lat1, long1)
print(lat2, long2)
print("")
lat_diff = lat1 - lat2
long_diff = long1 - long2
print("Lat diff", f'{lat_diff:.7f}')
print("Long diff", f'{long_diff:.7f}')

In [ ]:
road_elevation_long_list = road_elevation_nodes["Longitude"].values
print(road_elevation_long_list)

road_elevation_lat_list = road_elevation_nodes["Latitude"].values
print(road_elevation_lat_list)

if 34.029860 in road_elevation_long_list:
    print("Is here")
if -11.4585340 in road_elevation_lat_list:
    print("Is here too")

In [ ]:
long_unique_arr = road_elevation_nodes["Longitude"].unique()
lat_unique_arr = road_elevation_nodes["Latitude"].unique()

print(len(long_unique_arr))
print(len(lat_unique_arr))

# Create Dict

---

In [ ]:
road_elevation_dict = {}

for index, row, in road_elevation_nodes.iterrows():
    lat_long_pair = (round(row["To Latitude"], 3), round(row["To Longitude"], 3))
    if lat_long_pair not in road_elevation_dict:
        road_elevation_dict[lat_long_pair] = [index]
    else: 
        road_elevation_dict[lat_long_pair].append(index)

road_elevation_dict

In [ ]:
for index, row in road_elevation_nodes.iterrows():
    lat_long_pair = (round(row["Latitude"], 3), round(row["Longitude"], 3))
    
    if lat_long_pair in road_elevation_dict:
        fill_dataframe = road_elevation_dict[lat_long_pair]
        data_list = road_elevation_nodes.iat[index, 9]
        data_list = np.append(data_list, fill_dataframe)
        road_elevation_nodes.iat[index, 9] = data_list
        
    

road_elevation_nodes

In [ ]:
def calculate_euclidean_dist(lat1, long1, lat2, long2): 
    return math.sqrt((lat2 - lat1)**2 + (long2 - long1)**2)
    
    

In [ ]:
class Node: 

    def __init__(self, id):
        self.id = id
        self.connections = {}

    def fill_connections(self, dataframe):
        connections_list = dataframe.iat[self.id, 9]
        self_lat, self_long = dataframe.iat[index, 0], dataframe.iat[index, 1]
        for connection in connections_list: 
            connection = int(connection)
            connect_lat, connect_long = dataframe.iat[connection, 0], dataframe.iat[connection, 1]
            dist = calculate_euclidean_dist(self_lat, self_long, connect_lat, connect_long)
            self.connections[connection] = dist

In [ ]:
class Graph:
    def __init__(self, dataframe):
        self.lat_long_nodes = {}
        for index, row in dataframe.iterrows():
            node_lat = row["Latitude"]
            node_long = row["Longitude"]
            node = Node(index)
            node.fill_connections(dataframe)
            self.lat_long_nodes[(node_lat, node_long)] = node
            

In [ ]:
class state_node():
    def __init__(self, id, node, curr_dist, remain_dist, total_dist)
        self.id = id
        self.graph_node = node
        self.curr_dist = curr_dist
        self.remain_dist = remain_dist
        self.total_dist = total_dist

In [ ]:
class priority_queue():
    def __init__(self):
        queue = []
    
    def pop(self):
        
    

In [ ]:
graph = Graph(road_elevation_nodes)

In [ ]:
def a_star_search(graph, start_node, end_node):
    
    
    

# More Testing

---

In [ ]:
for index, row in road_elevation_nodes.iterrows(): 
    if row["Distance"] == 0.00125705485326:
        print(index)
    if row["Distance"] == 0.00129511125876:
        print(index)
    if row["Distance"] == 0.00035137504829:
        print(index)
    if row["Distance"] == 0.00024823242858:
        print(index)


In [ ]:
road_elevation_nodes.iloc[37757]
road_elevation_nodes.iloc[37757, 4]

In [ ]:
road_elevation_nodes.iloc[37758]
road_elevation_nodes.iloc[37758, 3]

In [ ]:
road_elevation_nodes.iloc[51658]
road_elevation_nodes.iloc[51658, 4]